In [1]:
import pandas as pd
import numpy as np
import dalex as dx

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Data exploration and pre-processing

We load data from `train-data.csv` and `X_test.csv` in pandas DataFrames.

In [2]:
train_data = pd.read_csv("./data/completed_train_data.csv.csv", sep=";")
X_set = pd.read_csv("./data/X_test.csv", sep=";")

We display `train_data` sorted by _index_.

In [3]:
train_data.sort_values(by="index")

,index,id_product,Region,Country,Site,Operations,Zone,Cluster,Reference proxy,Product Line proxy,Division proxy,Customer Persona proxy,Strategic Product Family proxy,Product Life cycel status,Date,Month 1,Month 2,Month 3,Month 4
2061569,0,0,EAJP,AU,AU_DC_Sydney,International Operations,Pacific,Australia,reference-1,Product Line-1,Division-1,Customer Segmentation-1,Strategic Product Family-1,NaN,sep-dec 2020,0,0,0,0
946355,0,0,EAJP,AU,AU_DC_Sydney,International Operations,Pacific,Australia,reference-1,Product Line-1,Division-1,Customer Segmentation-1,Strategic Product Family-1,NaN,sep-dec 2020,0,0,0,0
1222945,2,2,EAJP,AU,AU_DC_Sydney,International Operations,Pacific,Australia,reference-2,Product Line-1,Division-1,Customer Segmentation-3,Strategic Product Family-1,EOL,sep-dec 2020,0,0,0,0
2165839,2,2,EAJP,AU,AU_DC_Sydney,International Operations,Pacific,Australia,reference-2,Product Line-1,Division-1,Customer Segmentation-3,Strategic Product Family-1,EOL,sep-dec 2020,0,0,0,0
2567627,3,3,EAJP,AU,AU_DC_Sydney,International Operations,Pacific,Australia,reference-2,Product Line-1,Division-1,Customer Segmentation-4,Strategic Product Family-1,EOL,sep-dec 2020,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2607229,2203709,244853,CHINA,CN,CN_DC_Wuhan,China Operations,China & HK,China,reference-22406,Product Line-4,Division-3,Customer Segmentation-13,Strategic Product Family-9,NaN,may-jul 2023,NaN,50,120,50
3079899,2203710,244854,CHINA,CN,CN_DC_Wuhan,China Operations,China & HK,China,reference-22406,Product Line-4,Division-3,Customer Segmentation-4,Strategic Product Family-9,NaN,may-jul 2023,NaN,85,210,0
651780,2203710,244854,CHINA,CN,CN_DC_Wuhan,China Operations,China & HK,China,reference-22406,Product Line-4,Division-3,Customer Segmentation-4,Strategic Product Family-9,NaN,may-jul 2023,NaN,85,210,0
1124213,2203712,244856,CHINA,HK,HK_DC_HongKong,China Operations,China & HK,Hong Kong & Macao,reference-9687,Product Line-4,Division-3,Customer Segmentation-70,Strategic Product Family-9,EOL,may-jul 2023,NaN,0,0,0


We see that some rows with the same index seems to be duplicates (same index). By grouping by _index_, we can confirm that :

In [4]:
train_data.groupby("index").nunique().max()

id_product                        1
Region                            1
Country                           1
Site                              1
Operations                        1
Zone                              1
Cluster                           1
Reference proxy                   1
Product  Line proxy               1
Division proxy                    1
Customer Persona proxy            1
Strategic Product Family proxy    1
Product Life cycel status         1
Date                              1
Month 1                           1
Month 2                           1
Month 3                           1
Month 4                           1
dtype: int64

That being said, we'll be able to drop duplicates by _index_. We can also display `X_set` sorted by _index_.

In [5]:
X_set.sort_values(by="index")

,index,id_product,Region,Country,Site,Operations,Zone,Cluster,Reference proxy,Product Line proxy,Division proxy,Customer Persona proxy,Strategic Product Family proxy,Product Life cycel status,Date,Month 1,Month 2,Month 3
23512,1,1,EAJP,AU,AU_DC_Sydney,International Operations,Pacific,Australia,reference-2,Product Line-1,Division-1,Customer Segmentation-2,Strategic Product Family-1,EOL,sep-dec 2020,0,0,0
248701,7,7,EAJP,AU,AU_DC_Sydney,International Operations,Pacific,Australia,reference-2,Product Line-1,Division-1,Customer Segmentation-8,Strategic Product Family-1,EOL,sep-dec 2020,1,0,0
177623,20,20,EAJP,AU,AU_DC_Melbourne,International Operations,Pacific,Australia,reference-4,Product Line-1,Division-1,Customer Segmentation-10,Strategic Product Family-1,NaN,sep-dec 2020,2,1,9
208094,27,27,EAJP,AU,AU_DC_Perth,International Operations,Pacific,Australia,reference-4,Product Line-1,Division-1,Customer Segmentation-10,Strategic Product Family-1,NaN,sep-dec 2020,0,3,1
81837,34,34,EAJP,AU,AU_DC_Sydney,International Operations,Pacific,Australia,reference-8,Product Line-1,Division-1,Customer Segmentation-13,Strategic Product Family-1,NPI,sep-dec 2020,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283407,2203686,244830,CHINA,CN,CN_DC_Wuhan,China Operations,China & HK,China,reference-22404,Product Line-4,Division-3,Customer Segmentation-16,Strategic Product Family-9,NaN,may-jul 2023,NaN,0,0
301262,2203690,244834,CHINA,CN,CN_DC_SHD,China Operations,China & HK,China,reference-4151,Product Line-3,Division-2,Customer Segmentation-16,Strategic Product Family-7,NaN,may-jul 2023,NaN,0,1
75874,2203695,244839,CHINA,HK,SG_DC_ASD,China Operations,China & HK,Hong Kong & Macao,reference-4152,Product Line-3,Division-2,Customer Segmentation-33,Strategic Product Family-7,NaN,may-jul 2023,NaN,0,0
369953,2203700,244844,CHINA,CN,CN_DC_Shanghai,China Operations,China & HK,China,reference-22405,Product Line-4,Division-3,Customer Segmentation-3,Strategic Product Family-9,NaN,may-jul 2023,NaN,0,0


It seems _index_ values in `X_set` differs from _index_ values in `train_data`. We can confirm that :

In [6]:
np.intersect1d(train_data["index"].unique(), X_set["index"].unique())

array([], dtype=int64)

Then we can remove drop duplicates in `train_data` and concatenate it with `X_set` :

In [7]:
completed_data = pd.concat([train_data.drop_duplicates("index"), X_set])

completed_data.sort_values(by="index")

,index,id_product,Region,Country,Site,Operations,Zone,Cluster,Reference proxy,Product Line proxy,Division proxy,Customer Persona proxy,Strategic Product Family proxy,Product Life cycel status,Date,Month 1,Month 2,Month 3,Month 4
946355,0,0,EAJP,AU,AU_DC_Sydney,International Operations,Pacific,Australia,reference-1,Product Line-1,Division-1,Customer Segmentation-1,Strategic Product Family-1,NaN,sep-dec 2020,0,0,0,0
23512,1,1,EAJP,AU,AU_DC_Sydney,International Operations,Pacific,Australia,reference-2,Product Line-1,Division-1,Customer Segmentation-2,Strategic Product Family-1,EOL,sep-dec 2020,0,0,0,NaN
1222945,2,2,EAJP,AU,AU_DC_Sydney,International Operations,Pacific,Australia,reference-2,Product Line-1,Division-1,Customer Segmentation-3,Strategic Product Family-1,EOL,sep-dec 2020,0,0,0,0
888927,3,3,EAJP,AU,AU_DC_Sydney,International Operations,Pacific,Australia,reference-2,Product Line-1,Division-1,Customer Segmentation-4,Strategic Product Family-1,EOL,sep-dec 2020,0,0,0,0
229400,4,4,EAJP,AU,AU_DC_Sydney,International Operations,Pacific,Australia,reference-2,Product Line-1,Division-1,Customer Segmentation-5,Strategic Product Family-1,EOL,sep-dec 2020,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617716,2203708,244852,CHINA,CN,CN_DC_Wuhan,China Operations,China & HK,China,reference-22406,Product Line-4,Division-3,Customer Segmentation-3,Strategic Product Family-9,NaN,may-jul 2023,NaN,0,5,0
2607229,2203709,244853,CHINA,CN,CN_DC_Wuhan,China Operations,China & HK,China,reference-22406,Product Line-4,Division-3,Customer Segmentation-13,Strategic Product Family-9,NaN,may-jul 2023,NaN,50,120,50
651780,2203710,244854,CHINA,CN,CN_DC_Wuhan,China Operations,China & HK,China,reference-22406,Product Line-4,Division-3,Customer Segmentation-4,Strategic Product Family-9,NaN,may-jul 2023,NaN,85,210,0
149253,2203711,244855,CHINA,HK,HK_DC_HongKong,China Operations,China & HK,Hong Kong & Macao,reference-9686,Product Line-4,Division-3,Customer Segmentation-70,Strategic Product Family-9,EOL,may-jul 2023,NaN,0,0,NaN


We can now take a look at lines sharing the same _id_product_ :

In [8]:
completed_data.groupby("Reference proxy").count()

,index,id_product,Region,Country,Site,Operations,Zone,Cluster,Product Line proxy,Division proxy,Customer Persona proxy,Strategic Product Family proxy,Product Life cycel status,Date,Month 1,Month 2,Month 3,Month 4
Reference proxy,,,,,,,,,,,,,,,,,,
reference-1,9,9,9,9,9,9,9,9,9,9,9,9,0,9,8,9,9,9
reference-10,54,54,54,54,54,54,54,54,54,54,54,54,0,54,48,54,54,46
reference-100,27,27,27,27,27,27,27,27,27,27,27,27,27,27,24,27,27,19
reference-1000,144,144,144,144,144,144,144,144,144,144,144,144,0,144,128,144,144,117
reference-10000,9,9,9,9,9,9,9,9,9,9,9,9,0,9,8,9,9,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
reference-9995,9,9,9,9,9,9,9,9,9,9,9,9,9,9,8,9,9,8
reference-9996,9,9,9,9,9,9,9,9,9,9,9,9,9,9,8,9,9,7
reference-9997,9,9,9,9,9,9,9,9,9,9,9,9,9,9,8,9,9,6


We see that many lines correspond to the same product at different quarters, so we'll aggregate that information in one-line.

In [9]:
completed_data.groupby("id_product").aggregate([])

Empty DataFrame
Columns: []
Index: []

In [10]:
X_set["index"].value_counts()

index
490548     1
1035647    1
691029     1
229686     1
1619434    1
          ..
571112     1
1477185    1
917892     1
1540060    1
181981     1
Name: count, Length: 440743, dtype: int64

array([], dtype=int64)

In [45]:
full_set = pd.concat([train_data, X_set])
full_set

,index,id_product,Region,Country,Site,Operations,Zone,Cluster,Reference proxy,Product Line proxy,Division proxy,Customer Persona proxy,Strategic Product Family proxy,Product Life cycel status,Date,Month 1,Month 2,Month 3,Month 4
0,645874,156160,EUROPE,DE,NL_DC_Venray,Europe Operations,DACH,Germany,reference-13523,Product Line-4,Division-3,Customer Segmentation-3,Strategic Product Family-12,NaN,may-aug 2021,0,0,0,0
1,469488,224631,CHINA,CN,CN_DC_Shanghai,China Operations,China & HK,China,reference-12778,Product Line-4,Division-3,Customer Segmentation-14,Strategic Product Family-9,NaN,jan-apr 2021,0,1,0,0
2,348904,104047,EUROPE,GB,NL_DC_HLD,Europe Operations,UK and Ireland,United Kingdom,reference-3513,Product Line-3,Division-2,Customer Segmentation-23,Strategic Product Family-7,NaN,jan-apr 2021,0,2,0,0
3,1725822,11823,EAJP,AU,AU_DC_Perth,International Operations,Pacific,Australia,reference-672,Product Line-1,Division-1,Customer Segmentation-11,Strategic Product Family-3,NaN,jan-apr 2023,0,0,0,0
4,404781,159924,EUROPE,ES,NL_DC_HLD,Europe Operations,Iberia,Spain,reference-3496,Product Line-3,Division-2,Customer Segmentation-14,Strategic Product Family-7,NaN,jan-apr 2021,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440738,849490,114919,EUROPE,IT,IT_DC_Venaria,Europe Operations,Italy,Italy,reference-6671,Product Line-4,Division-3,Customer Segmentation-15,Strategic Product Family-9,NaN,sep-dec 2021,0,2,0,NaN
440739,1199858,220430,CHINA,CN,CN_MF_SSPA,China Operations,China & HK,China,reference-19181,Product Line-5,Division-3,Customer Segmentation-13,Strategic Product Family-15,NaN,jan-apr 2022,0,0,0,NaN
440740,1007643,28215,EAJP,AU,AU_DC_Brisbane,International Operations,Pacific,Australia,reference-1966,Product Line-1,Division-1,Customer Segmentation-6,Strategic Product Family-4,NaN,jan-apr 2022,0,0,0,NaN
440741,1647728,178586,EUROPE,PL,HU_DC_CEELog_Budapest,Europe Operations,CEEI,Middle Eastern Europe,reference-7364,Product Line-4,Division-3,Customer Segmentation-14,Strategic Product Family-9,NaN,sep-dec 2022,2,0,0,NaN


In [ ]:
train_data.groupby("index").nunique()

In [5]:
train_data.drop_duplicates("index").shape

(1762970, 19)

In [6]:
train_data = train_data.drop_duplicates("index")
train_data

,index,id_product,Region,Country,Site,Operations,Zone,Cluster,Reference proxy,Product Line proxy,Division proxy,Customer Persona proxy,Strategic Product Family proxy,Product Life cycel status,Date,Month 1,Month 2,Month 3,Month 4
0,645874,156160,EUROPE,DE,NL_DC_Venray,Europe Operations,DACH,Germany,reference-13523,Product Line-4,Division-3,Customer Segmentation-3,Strategic Product Family-12,NaN,may-aug 2021,0,0,0,0
1,469488,224631,CHINA,CN,CN_DC_Shanghai,China Operations,China & HK,China,reference-12778,Product Line-4,Division-3,Customer Segmentation-14,Strategic Product Family-9,NaN,jan-apr 2021,0,1,0,0
2,348904,104047,EUROPE,GB,NL_DC_HLD,Europe Operations,UK and Ireland,United Kingdom,reference-3513,Product Line-3,Division-2,Customer Segmentation-23,Strategic Product Family-7,NaN,jan-apr 2021,0,2,0,0
3,1725822,11823,EAJP,AU,AU_DC_Perth,International Operations,Pacific,Australia,reference-672,Product Line-1,Division-1,Customer Segmentation-11,Strategic Product Family-3,NaN,jan-apr 2023,0,0,0,0
4,404781,159924,EUROPE,ES,NL_DC_HLD,Europe Operations,Iberia,Spain,reference-3496,Product Line-3,Division-2,Customer Segmentation-14,Strategic Product Family-7,NaN,jan-apr 2021,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173317,1445304,221019,CHINA,CN,CN_DC_SHD,China Operations,China & HK,China,reference-17197,Product Line-2,Division-2,Customer Segmentation-28,Strategic Product Family-5,NaN,may-aug 2022,0,0,0,0
3173331,738575,4004,EAJP,AU,AU_DC_Perth,International Operations,Pacific,Australia,reference-309,Product Line-1,Division-1,Customer Segmentation-8,Strategic Product Family-3,EOL,sep-dec 2021,0,1,0,0
3173333,68026,68026,NAM,US,US_DC_Mechanicsburg,North America Operations,US,USA,reference-7393,Product Line-4,Division-3,Customer Segmentation-4,Strategic Product Family-9,NaN,sep-dec 2020,18,1,0,0
3173334,1841243,127244,EUROPE,IT,IT_DC_Venaria,Europe Operations,Italy,Italy,reference-8620,Product Line-4,Division-3,Customer Segmentation-4,Strategic Product Family-12,NaN,jan-apr 2023,0,1,0,0


In [3]:
train_data.groupby("id_product").nunique()

,index,Region,Country,Site,Operations,Zone,Cluster,Reference proxy,Product Line proxy,Division proxy,Customer Persona proxy,Strategic Product Family proxy,Product Life cycel status,Date,Month 1,Month 2,Month 3,Month 4
id_product,,,,,,,,,,,,,,,,,,
0,9,1,1,1,1,1,1,1,1,1,1,1,0,9,2,1,1,1
1,6,1,1,1,1,1,1,1,1,1,1,1,1,6,1,2,1,1
2,7,1,1,1,1,1,1,1,1,1,1,1,1,7,1,1,2,1
3,7,1,1,1,1,1,1,1,1,1,1,1,1,7,1,2,2,2
4,7,1,1,1,1,1,1,1,1,1,1,1,1,7,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244852,9,1,1,1,1,1,1,1,1,1,1,1,0,9,1,1,2,1
244853,9,1,1,1,1,1,1,1,1,1,1,1,0,9,2,4,6,4
244854,7,1,1,1,1,1,1,1,1,1,1,1,0,7,3,4,5,3


In [ ]:
train_data.groupby()

In [8]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.tree import DecisionTreeClassifier

X = train_data.drop(columns="Month 4")
y = train_data["Month 4"]

categorical_features    = [
                            "Region",
                            "Country",
                            "Site",
                            "Operations",
                            "Zone",
                            "Cluster",
                            "Reference proxy",
                            "Product  Line proxy", #typo !
                            "Division proxy",
                            "Customer Persona proxy",
                            "Strategic Product Family proxy",
                            "Product Life cycel status", #typo !
                            "Date"
                        ]

numerical_features      = [
                            "index",
                            "id_product",
                            "Month 1",
                            "Month 2",
                            "Month 3",
                        ]

# Pipeline de transformation des features catégoriques 

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist')),
    #('ordinal', OrdinalEncoder())
])

# Pipeline de transformation des features numériques 

numerical_transformer = Pipeline(steps=[
    ('remove_space', FunctionTransformer(
        np.frompyfunc(lambda s: str(s).replace(" ", ""), 1, 1) #
    )),
    ('scaler', StandardScaler()),
    ('imputer', SimpleImputer(strategy='constant', fill_value=-1))
    #TODO transformer for the NaN ?
])

preprocessor = ColumnTransformer(transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
])



clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(max_depth=7, random_state=123))
])

clf.fit(X, y)

#preprocessor.fit(X)

KeyboardInterrupt: 

In [79]:
preprocessor.transform(X)

<1762970x22682 sparse matrix of type '<class 'numpy.float64'>'
	with 31733460 stored elements in Compressed Sparse Row format>